### This notebook is a modified python version of [this R notebook](https://www.kaggle.com/oxzplvifi/tabular-residual-network) from @oxzplvifi

In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import pandas as pd       
import matplotlib as mat
import matplotlib.pyplot as plt    
import numpy as np
import seaborn as sns
%matplotlib inline

import random
import os
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler


from tensorflow import keras
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import callbacks

In [ ]:
#Trying to get reproducible results
from numpy.random import seed
seed(42)
from tensorflow.random import set_seed
set_seed(42)

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv').drop('id', axis = 1)
df_train['target'] = df_train['target'].str.slice(start=6).astype(int) - 1

y_train = df_train['target'].copy()
X_train = df_train.copy().drop('target', axis = 1)

X_test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv').drop('id', axis = 1)

In [ ]:
#Converting target series to matrix for multiclass classification on Keras

Y_train = to_categorical(y_train)
Y_train

## Creating and Evaluating the NN

In [ ]:
def get_model(X_train, n_classes = 9):
    inputs = layers.Input(shape = (X_train.shape[1],))
    
    embed = layers.Embedding(360, 8)(inputs)
    embed = layers.Flatten()(embed)
    
    hidden = layers.Dropout(0.2)(embed)
    hidden = tfa.layers.WeightNormalization(layers.Dense(units=128, activation='selu', kernel_initializer="lecun_normal"))(hidden)
    
    output = layers.Dropout(0.2)(layers.Concatenate()([embed, hidden]))
    output = tfa.layers.WeightNormalization(layers.Dense(units=64, activation='relu'))(output) 
    
    output = layers.Dropout(0.3)(layers.Concatenate()([embed, hidden, output]))
    output = tfa.layers.WeightNormalization(layers.Dense(units=32, activation='elu'))(output) 
    output = layers.Dense(n_classes, activation = 'softmax')(output)
    
    model = keras.Model(inputs=inputs, outputs=output, name="res_nn_model")
    
    return model

In [ ]:
K.clear_session()
cce = keras.losses.CategoricalCrossentropy(name = 'cat_crossentr')

def custom_metric(y_true, y_pred):
    y_pred = K.clip(y_pred, 1e-15, 1-1e-15)
    loss = cce(y_true, y_pred)
    return loss

In [ ]:
OOF_PRED = np.zeros((X_train.shape[0], 9))
TEST_PRED = np.zeros((X_test.shape[0], 9))
N_starts = 10
for i in range(N_starts):
    N = 10
    kf = StratifiedKFold(n_splits=N, shuffle=True, random_state=13 + i)

    for FOLD, (trn_idx, val_idx) in enumerate(kf.split(y_train, y_train)):
        X_tr, y_tr = X_train.iloc[trn_idx, :], Y_train[trn_idx, :]
        X_val, y_val = X_train.iloc[val_idx, :], Y_train[val_idx, :]

        K.clear_session()
        model = get_model(X_tr)
        model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(learning_rate=2e-4), metrics=custom_metric)

        early_stopping = callbacks.EarlyStopping(patience=10, min_delta=1e-5, restore_best_weights=True)
        plateau = callbacks.ReduceLROnPlateau(factor = 0.7, patience = 2, verbose = 0) 

        model.fit(X_tr, y_tr,
              batch_size = 256, epochs = 100,
              validation_data=(X_val, y_val),
              callbacks=[early_stopping, plateau],
              verbose = 0)

        pred = model.predict(X_val)
        OOF_PRED[val_idx, :] += pred / N_starts
        TEST_PRED += model.predict(X_test) / N / N_starts

        nn_logloss = log_loss(y_train[val_idx], pred)

        print(f"START {i:d} FOLD {FOLD:d}: logloss score {nn_logloss:.6f}")

        del model
        _ = gc.collect()

nn_logloss = log_loss(y_train, OOF_PRED)
print(f"full logloss score {nn_logloss}")

## Making Predictions

In [ ]:
ss = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')
ss

In [ ]:
ss.iloc[:, 1:] = TEST_PRED
ss

In [ ]:
ss.to_csv('nn_predict.csv', index = False)

In [ ]:
oof_preds_df = pd.DataFrame(OOF_PRED, columns = ['Class_' + str(i) for i in range(1, 10)])
oof_preds_df.insert(0, 'id', list(range(oof_preds_df.shape[0])))
oof_preds_df

In [ ]:
oof_preds_df.to_csv('OOF_nn_predict.csv', index = False)